In [3]:
import numpy as np

def haar_wavelet_transform(signal):
    length = len(signal)
    if length % 2 != 0:
        raise ValueError("Input signal length must be even for Haar wavelet transform.")

    h = np.array([1, 1]) / np.sqrt(2)
    g = np.array([1, -1]) / np.sqrt(2)

    # Forward transform
    approx = np.convolve(signal, h, mode='valid')[::2]
    detail = np.convolve(signal, g, mode='valid')[::2]

    return approx, detail

def inverse_haar_wavelet_transform(approx, detail):
    h = np.array([1, 1]) / np.sqrt(2)
    g = np.array([1, -1]) / np.sqrt(2)

    # Upsample and convolve to reconstruct
    approx_upsampled = np.zeros(2 * len(approx))
    detail_upsampled = np.zeros(2 * len(detail))

    approx_upsampled[::2] = approx
    detail_upsampled[::2] = detail

    signal_reconstructed = np.convolve(approx_upsampled, h, mode='full')[:-1] + np.convolve(detail_upsampled, g, mode='full')[:-1]

    return signal_reconstructed

def haar_wavelet_2d_transform(image):
    rows, cols = image.shape
    approx_coeffs = np.zeros_like(image, dtype=float)
    detail_coeffs = np.zeros_like(image, dtype=float)

    for i in range(rows):
        approx_row, detail_row = haar_wavelet_transform(image[i, :])
        approx_coeffs[i, :len(approx_row)] = approx_row
        detail_coeffs[i, :len(detail_row)] = detail_row

    for j in range(cols):
        approx_col, detail_col = haar_wavelet_transform(approx_coeffs[:, j])
        approx_coeffs[:len(approx_col), j] = approx_col
        detail_coeffs[:len(detail_col), j] = detail_col

    return approx_coeffs, detail_coeffs

def inverse_haar_wavelet_2d_transform(approx_coeffs, detail_coeffs):
    rows, cols = approx_coeffs.shape
    reconstructed_image = np.zeros_like(approx_coeffs, dtype=float)

    for j in range(cols):
        approx_col = approx_coeffs[:, j]
        detail_col = detail_coeffs[:, j]

        reconstructed_col = inverse_haar_wavelet_transform(approx_col, detail_col)
        reconstructed_image[:, j] = reconstructed_col[:rows]  # Adjust shape

    for i in range(rows):
        approx_row = reconstructed_image[i, :]
        detail_row = detail_coeffs[i, :]

        reconstructed_row = inverse_haar_wavelet_transform(approx_row, detail_row)
        reconstructed_image[i, :] = reconstructed_row[:cols]  # Adjust shape

    return reconstructed_image

# Example usage
original_image = np.random.random((512, 512))
approx_coeffs, detail_coeffs = haar_wavelet_2d_transform(original_image)
reconstructed_image = inverse_haar_wavelet_2d_transform(approx_coeffs, detail_coeffs)


In [4]:
print(original_image)

[[6.24023349e-01 3.15461508e-01 2.14006193e-01 ... 8.82481919e-01
  7.79420394e-01 7.99767319e-01]
 [3.77284493e-01 2.25746333e-01 3.34271519e-01 ... 7.34718363e-02
  4.52309410e-01 2.01093427e-01]
 [8.78530449e-01 2.67981630e-01 7.40358278e-01 ... 8.51699414e-01
  1.35962317e-04 2.94775598e-01]
 ...
 [2.90069324e-01 7.69977214e-01 4.69166291e-01 ... 1.32808497e-01
  1.55471711e-01 7.28742251e-01]
 [1.40993217e-01 6.78607734e-01 4.93503821e-01 ... 8.07397003e-01
  3.48022096e-01 4.41742948e-01]
 [2.17872018e-01 8.94008472e-02 4.63359862e-01 ... 6.20878396e-01
  8.07021154e-01 2.46888911e-01]]


In [7]:
print(detail_coeffs.shape)

(512, 512)
